In [1]:
# install anvil and import anvil server
!pip install anvil-uplink
import anvil.server

In [2]:
!pip install PyMySQL
import pymysql

In [3]:
!pip install plotly==5.6.0
import datetime
import plotly.express as px
import pandas as pd

In [4]:
# Connect to the anvil frontend
anvil.server.connect("PVI2LD3VLGMNLT22SC6MILQG-FX3COW6HW5E3NGBT")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [5]:
def set_connection():
    db = pymysql.connect(host='sh-cynosdbmysql-grp-75qwodo8.sql.tencentcdb.com',
                         port =29563,user='capstone',password='capstone123!!!',
                         db='circle_database_version3')
    return db

In [6]:
def stop_connection(db):
    db.close()

In [7]:
def generate_query(vehicle_id, start_time, end_time, lower_constraint, upper_constraint, table, kind):
    query = "SELECT " + kind + " , elapsed_time FROM " + table + " WHERE elapsed_time > '" + str(start_time) + "' and elapsed_time < '" + str(end_time) + "' and drive_id = '" + str(vehicle_id) + "' and " + kind + " > " + lower_constraint + " and " + kind + " < " + upper_constraint
    return query

In [8]:
def datetime_to_float(d):
    return d.timestamp()

In [9]:
@anvil.server.callable
def query_lead_distance(vehicle_id, start_time, end_time, lower_lead, upper_lead):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    MAXIMUM = 10000
    lower_lead = str(lower_lead) if lower_lead != "" else str(- MAXIMUM)        
    upper_lead = str(upper_lead) if upper_lead != "" else str(MAXIMUM)  
        
    query = generate_query(vehicle_id, start_time, end_time, lower_lead, upper_lead, "fact_lead_distance", "lead_distance")
    db = set_connection()  
    cursor = db.cursor()
    cursor.execute(query)
    data = cursor.fetchall()

    stop_connection(db)
    return list(data)

@anvil.server.callable
def query_speed(vehicle_id, start_time, end_time, lower_speed, upper_speed):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    MAXIMUM = 10000
    lower_speed = str(lower_speed) if lower_speed != "" else str(- MAXIMUM)        
    upper_speed = str(upper_speed) if upper_speed != "" else str(MAXIMUM)  
        
    query = generate_query(vehicle_id, start_time, end_time, lower_speed, upper_speed, "fact_speed", "speed")
    
    db = set_connection()  
    
    cursor = db.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    
    stop_connection(db)
    
    return list(data)

@anvil.server.callable
def query_acc(vehicle_id, start_time, end_time, lower_acceleration, upper_acceleration):
    st = start_time.replace(tzinfo=None)
    et = end_time.replace(tzinfo=None)
    MAXIMUM = 10000 
    lower_acceleration = str(lower_acceleration) if lower_acceleration != "" else str(- MAXIMUM)        
    upper_acceleration = str(upper_acceleration) if upper_acceleration != "" else str(MAXIMUM)
    query = generate_query(vehicle_id, start_time, end_time, lower_acceleration, upper_acceleration, "fact_acc", "fact_acc")
    
    db = set_connection() 
    
    cursor = db.cursor()
    cursor.execute(query)
    data = cursor.fetchall()

    stop_connection(db)
    return list(data)

In [10]:
@anvil.server.callable
def query_all(vehicle_id, start_time, end_time):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    start_time = datetime_to_float(start_time)
    end_time = datetime_to_float(end_time)
    
    MAXIMUM = 10000
    lower_lead = str(- MAXIMUM)        
    upper_lead = str(MAXIMUM)  
    lower_acceleration = str(- MAXIMUM)        
    upper_acceleration = str(MAXIMUM)
    lower_speed = str(- MAXIMUM)        
    upper_speed = str(MAXIMUM) 
        
    speed_query = generate_query(int(vehicle_id), start_time, end_time, lower_speed, upper_speed, "fact_speed", "speed")
    acceleration_query = generate_query(int(vehicle_id), start_time, end_time, lower_acceleration, upper_acceleration, "fact_acceleration", "acceleration")
    lead_query = generate_query(int(vehicle_id), start_time, end_time, lower_lead, upper_lead, "fact_lead_distance", "lead_distance")
    db = set_connection()  
    cursor = db.cursor()
    cursor.execute(acceleration_query)
    data1 = cursor.fetchall()

    cursor = db.cursor()
    cursor.execute(speed_query)
    data2 = cursor.fetchall()
    
    cursor = db.cursor()
    cursor.execute(lead_query)
    data3 = cursor.fetchall()
    stop_connection(db)
    return list(data1), list(data2), list(data3)

In [25]:
@anvil.server.callable
def query_drive_id_by_gps(lat, long):
    RANGE = 10000
    query_id = "Select drive_id FROM fact_gps_message where latitude > " + str(float(lat)-RANGE) + " and latitude < " + str(float(lat)+RANGE) + " and longitude < " + str(float(long)+RANGE) + " and longitude > " + str(float(long)-RANGE)
    
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(set(cursor.fetchall()))
    string_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vehicle_id FROM dim_drive WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vehicle_id = cursor.fetchall()[0][0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(vehicle_id)
        cursor.execute(query)
        vin = cursor.fetchall()[0][0]
        string_list.append("Vehicle ID: " + str(vin) + " Drive ID: " + str(cleaned_drive_id))
    stop_connection(db)
    return string_list

In [26]:
@anvil.server.callable
def query_routine(drive_id):
    query_gps = "Select latitude, longitude FROM fact_gps_message where drive_id = " + str(drive_id)
    
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_gps)
    gps_data = cursor.fetchall()
    
    lat = []
    long = []
    for latlong in gps_data:
        lat.append(latlong[0])
        long.append(latlong[1])
        
    stop_connection(db)
    return lat, long

In [27]:
@anvil.server.callable
def query_drive_id_by_time(start_time, end_time):
    start_time = start_time.replace(tzinfo=None)
    end_time = end_time.replace(tzinfo=None)
    start_time = datetime_to_float(start_time)
    end_time = datetime_to_float(end_time)
    query_id = "Select vehicle_id FROM dim_drive WHERE systime_start > " + str(start_time) + " and systime_end < " + str(end_time)
    db = set_connection() 
    cursor = db.cursor()
    cursor.execute(query_id)
    drive_ids = list(cursor.fetchall())
    string_list = []
    for drive_id in drive_ids:
        cleaned_drive_id = drive_id[0]
        query = "Select vin FROM dim_vehicle WHERE id = " + str(cleaned_drive_id)
        cursor.execute(query)
        vin = cursor.fetchall()[0][0]
        string_list.append("Vehicle ID: " + str(vin) + " Drive ID: " + str(cleaned_drive_id))
    return string_list

In [ ]:
anvil.server.wait_forever()

(1,)
1
2T3W1RFVXKW033343
(2,)
2
2T3Y1RFV8KC014025
['Vehicle ID: 2T3W1RFVXKW033343 Drive ID: 1', 'Vehicle ID: 2T3Y1RFV8KC014025 Drive ID: 2']
(1,)
1
2T3W1RFVXKW033343
(2,)
2
2T3Y1RFV8KC014025
['Vehicle ID: 2T3W1RFVXKW033343 Drive ID: 1', 'Vehicle ID: 2T3Y1RFV8KC014025 Drive ID: 2']
